In [11]:
import os
import glob
import pandas as pd
from PIL import Image
from IPython.display import HTML

def to_dataframe(folder_path):
    data = []
    for file_path in glob.glob(os.path.join(folder_path, '*')):
        try:
            with Image.open(file_path) as img:
                file_info = {
                    "name": os.path.basename(file_path),                  
                    "color model": img.mode,                               
                    "format": img.format,                                  
                    "channels": len(img.getbands()),                      
                    "size (MB)": round(os.path.getsize(file_path) / (1024 * 1024), 2), 
                    "width": img.width,                                    
                    "height": img.height,                                 
                    "full path": file_path,                               
                    "image": f"<img src='{file_path}' width='100'>"        
                }
                data.append(file_info)
        except IOError:
            continue
    
    df = pd.DataFrame(data)
    return df
folder_path = r'C:\Users\opapo\PycharmProjects\pythonProject6\images'
df = to_dataframe(folder_path)

HTML(df.to_html(escape=False))

,name,color model,format,channels,size (MB),width,height,full path,image
0,image1.jpg,RGB,JPEG,3,0.26,1600,1014,C:\Users\opapo\PycharmProjects\pythonProject6\images\image1.jpg,
1,image10.jpg,RGB,JPEG,3,0.08,1280,1280,C:\Users\opapo\PycharmProjects\pythonProject6\images\image10.jpg,
2,image11.jpg,RGB,JPEG,3,0.13,1280,720,C:\Users\opapo\PycharmProjects\pythonProject6\images\image11.jpg,
3,image12.jpg,RGB,JPEG,3,0.09,1200,720,C:\Users\opapo\PycharmProjects\pythonProject6\images\image12.jpg,
4,image13.jpg,RGB,JPEG,3,0.06,659,577,C:\Users\opapo\PycharmProjects\pythonProject6\images\image13.jpg,
5,image14.jpg,RGB,JPEG,3,0.03,560,330,C:\Users\opapo\PycharmProjects\pythonProject6\images\image14.jpg,
6,image2.jpg,RGB,JPEG,3,0.03,474,340,C:\Users\opapo\PycharmProjects\pythonProject6\images\image2.jpg,
7,image3.jpg,RGB,JPEG,3,0.27,1574,1180,C:\Users\opapo\PycharmProjects\pythonProject6\images\image3.jpg,
8,image4.jpg,RGB,JPEG,3,0.06,1200,675,C:\Users\opapo\PycharmProjects\pythonProject6\images\image4.jpg,
9,image5.jpg,RGB,JPEG,3,0.02,323,243,C:\Users\opapo\PycharmProjects\pythonProject6\images\image5.jpg,


In [ ]:
import os
import glob
from PIL import Image, ImageEnhance
import random

def crop_to_square(img):
    width, height = img.size
    side = min(width, height)
    left = (width - side) / 2
    top = (height - side) / 2
    right = (width + side) / 2
    bottom = (height + side) / 2
    return img.crop((left, top, right, bottom))

def apply_random_filter(img):
    filter_color = random.choice(['red', 'green', 'blue', 'yellow', 'magenta', 'cyan'])
    overlay = Image.new('RGB', img.size, color=filter_color)
    return Image.blend(img.convert('RGB'), overlay, alpha=0.3)

def create_poster(folder_path, output_name, images_per_row, images_per_col):
    images_paths = glob.glob(os.path.join(folder_path, '*'))
    
    square_size = 500
    
    poster_width = images_per_row * square_size
    poster_height = images_per_col * square_size
    poster = Image.new('RGB', (poster_width, poster_height), "white")
    
    for row in range(images_per_col):
        for col in range(images_per_row):
            img = None
            attempts = 0
          
            while img is None and attempts < 5:
                img_path = random.choice(images_paths)
                try:
                    img = Image.open(img_path)
                    
                    img = img.convert("RGB")
                 
                    img = crop_to_square(img)
                    img = img.resize((square_size, square_size))
                 
                    img = apply_random_filter(img)
                except (IOError, OSError):
                    print(f"Помилка відкриття файлу {img_path}.")
                    img = None
                    attempts += 1
        
            if img is not None:
                x = col * square_size
                y = row * square_size
                poster.paste(img, (x, y))
            else:
                print(f"Не вдалося знайти потрібне зображення ({row}, {col}).")
    
    output_path = os.path.join(folder_path, output_name)
    poster.save(output_path)
    print(f"Постер сохранён как {output_path}")

folder_path = r'C:\Users\opapo\PycharmProjects\pythonProject6\images'

create_poster(folder_path, 'poster1.jpg', 4, 3)  
create_poster(folder_path, 'poster2.jpg', 5, 2)  
create_poster(folder_path, 'poster3.jpg', 3, 3)  
create_poster(folder_path, 'poster4.jpg', 4, 4)  